In [154]:
import pandas as pd
import glob
from datetime import datetime
import numpy as np# set search path and glob for files
# here we want to look for csv files in the input directory
path = r'C:\Users\Salih\Desktop\Projects\Doğuş Datathon 2021'
files = glob.glob(path + '/*.csv')

# create empty list to store dataframes
li = []

# loop through list of files and read each one into a dataframe and append to list
for f in files:
    # read in csv
    temp_df = pd.read_csv(f)
    # append df to list
    li.append(temp_df)
    print(f'Successfully created dataframe for {f} with shape {temp_df.shape}')

Successfully created dataframe for C:\Users\Salih\Desktop\Projects\Doğuş Datathon 2021\EUR_TRY Geçmiş Verileri.csv with shape (6688, 6)
Successfully created dataframe for C:\Users\Salih\Desktop\Projects\Doğuş Datathon 2021\FINAL_CUSTOMER_DATATHON.csv with shape (849919, 10)
Successfully created dataframe for C:\Users\Salih\Desktop\Projects\Doğuş Datathon 2021\FINAL_CUSTOMER_RELATED_TABLE_FOR_DATATHON.csv with shape (374879, 7)
Successfully created dataframe for C:\Users\Salih\Desktop\Projects\Doğuş Datathon 2021\FINAL_SALES_FILE_DATATHON.csv with shape (397526, 7)
Successfully created dataframe for C:\Users\Salih\Desktop\Projects\Doğuş Datathon 2021\FINAL_SIFIR_ARAC_ALANLAR_DATATHON.csv with shape (169525, 4)
Successfully created dataframe for C:\Users\Salih\Desktop\Projects\Doğuş Datathon 2021\FINAL_VEHICLE_TABLE_DATATHON.csv with shape (345092, 8)
Successfully created dataframe for C:\Users\Salih\Desktop\Projects\Doğuş Datathon 2021\MASK_SERVIS_BAKIM_DATATHON_FINAL.csv with shape (10

In [155]:
euro_try_data = li[0]
final_cust = li[1]
final_cust_related = li[2]
final_sales = li[3]
final_sifir_arac = li[4]
final_vehicle = li[5]
mask_serv_bakim = li[6]
sample_sub = li[7]

In [156]:
euro_try_data['Şimdi'] = euro_try_data['Şimdi'].apply(lambda x: int(x.replace(',', '')))

In [157]:
euro_try_data["Şimdi"]=euro_try_data["Şimdi"].astype(np.int32)

In [158]:
euro_try_data["Şimdi"]=euro_try_data["Şimdi"]/10000

In [159]:
euro_try_data['Tarih'] = euro_try_data['Tarih'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y-%m-%d'))

In [160]:
euro_try_data['Ay'] = euro_try_data['Tarih'].apply(lambda x: int(x.split("-")[1]))

In [161]:
euro_try_data['Yıl'] = euro_try_data['Tarih'].apply(lambda x: int(x.split("-")[0]))

In [162]:
euro_try_data

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Fark %,Ay,Yıl
0,2021-08-31,9.8051,"98,867","99,243","97,880","-0,74%",8,2021
1,2021-08-30,9.8782,"98,515","99,083","98,222","0,39%",8,2021
2,2021-08-27,9.8394,"98,629","99,047","98,125","-0,15%",8,2021
3,2021-08-26,9.8541,"98,636","98,947","98,049","0,04%",8,2021
4,2021-08-25,9.8501,"98,855","99,084","98,052","-0,18%",8,2021
...,...,...,...,...,...,...,...,...
6683,1996-01-08,0.0000,"0,0000","0,0000","0,0000","0,00%",1,1996
6684,1996-01-05,0.0000,"0,0000","0,0000","0,0000","0,00%",1,1996
6685,1996-01-04,0.0000,"0,0000","0,0000","0,0000","0,00%",1,1996
6686,1996-01-03,0.0000,"0,0000","0,0000","0,0000","0,00%",1,1996


FINAL_CUSTOMER_DATATHON
- FK_ADDRESS_COMMUNICATION_CITY bazı değerler outlier, drop or replace?
    ist > istanbul , asd, x, y, z, L etc.
- BIRTH_DATE outlierlar var
    araç sahipliği yaş sınırı nedir? ondan küçük olan değerleri boşla değiştir

In [163]:
prev_month = euro_try_data.groupby(['Yıl', 'Ay'])['Şimdi'].mean().reset_index()

In [164]:
prev_month.rename(columns={"Şimdi":"Last_month_mean"}, inplace=True)

In [165]:
prev_month["Last_month_std"]=euro_try_data.groupby(['Yıl', 'Ay'])['Şimdi'].std().reset_index()["Şimdi"]

In [166]:
euro_try_data["Tarih"] = euro_try_data["Tarih"].astype("datetime64")

In [167]:
euro_try_data=euro_try_data.sort_values(by=["Tarih"], ascending=True).reset_index()

In [168]:
prev_month['last3_avg'] = prev_month['Last_month_mean'].rolling(3).mean()

In [169]:
prev_month

,Yıl,Ay,Last_month_mean,Last_month_std,last3_avg
0,1996,1,0.000000,0.000000,NaN
1,1996,2,0.000000,0.000000,NaN
2,1996,3,0.000000,0.000000,0.000000
3,1996,4,0.000000,0.000000,0.000000
4,1996,5,0.000000,0.000000,0.000000
...,...,...,...,...,...
303,2021,4,9.765923,0.166538,9.157386
304,2021,5,10.175371,0.147156,9.690865
305,2021,6,10.377268,0.095515,10.106187
306,2021,7,10.153195,0.092964,10.235278


In [170]:
euro_try_data["three_month_std"] = euro_try_data['Şimdi'].rolling(90).std()

In [175]:
threemonth = euro_try_data.groupby(['Yıl', 'Ay'])['Tarih'].tail(1).sort_values(ascending=True).reset_index(drop=True)

In [176]:
pd.DataFrame(threemonth)

,Tarih
0,1996-01-31
1,1996-02-29
2,1996-03-29
3,1996-04-30
4,1996-05-31
...,...
303,2021-04-30
304,2021-05-31
305,2021-06-30
306,2021-07-30


In [177]:
last_df = pd.merge(threemonth, euro_try_data, on=['Tarih'], how='left')

In [178]:
last_df

,Tarih,index,Şimdi,Açılış,Yüksek,Düşük,Fark %,Ay,Yıl,three_month_std
0,1996-01-31,6666,0.0000,"0,0000","0,0000","0,0000","0,00%",1,1996,NaN
1,1996-02-29,6645,0.0000,"0,0000","0,0000","0,0000","0,00%",2,1996,NaN
2,1996-03-29,6624,0.0000,"0,0000","0,0000","0,0000","0,00%",3,1996,NaN
3,1996-04-30,6602,0.0000,"0,0000","0,0000","0,0000","0,00%",4,1996,NaN
4,1996-05-31,6579,0.0000,"0,0000","0,0000","0,0000","0,00%",5,1996,0.000000
...,...,...,...,...,...,...,...,...,...,...
303,2021-04-30,87,9.9431,"99,646","100,458","99,058","-0,07%",4,2021,0.459533
304,2021-05-31,66,10.3678,"104,334","104,762","103,126","-0,51%",5,2021,0.631896
305,2021-06-30,44,10.3182,"103,958","104,190","102,581","-0,64%",6,2021,0.534640
306,2021-07-30,22,10.0115,"100,486","100,917","99,569","-0,35%",7,2021,0.264689


In [179]:
last_df

,Yıl,Ay,Last_month_mean,Last_month_std,last3_avg
0,1996,1,0.000000,0.000000,NaN
1,1996,2,0.000000,0.000000,NaN
2,1996,3,0.000000,0.000000,0.000000
3,1996,4,0.000000,0.000000,0.000000
4,1996,5,0.000000,0.000000,0.000000
...,...,...,...,...,...
303,2021,4,9.765923,0.166538,9.157386
304,2021,5,10.175371,0.147156,9.690865
305,2021,6,10.377268,0.095515,10.106187
306,2021,7,10.153195,0.092964,10.235278


In [186]:
df1 = pd.merge(last_df, prev_month, left_index=True, right_index=True)

In [187]:
df1 = df1[["Tarih","Yıl_y","Ay_y","Last_month_mean","Last_month_std","last3_avg","three_month_std"]]

In [191]:
df11 = df1[df1["Tarih"]>"2009-12-31"]

In [192]:
df11

,Tarih,Yıl_y,Ay_y,Last_month_mean,Last_month_std,last3_avg,three_month_std
168,2010-01-29,2010,1,2.103667,0.016375,2.176349,0.050958
169,2010-02-26,2010,2,2.077905,0.013539,2.126309,0.066233
170,2010-03-31,2010,3,2.081848,0.017102,2.087806,0.061150
171,2010-04-30,2010,4,2.002650,0.024501,2.054134,0.046440
172,2010-05-31,2010,5,1.949257,0.031798,2.011252,0.059950
...,...,...,...,...,...,...,...
303,2021-04-30,2021,4,9.765923,0.166538,9.157386,0.459533
304,2021-05-31,2021,5,10.175371,0.147156,9.690865,0.631896
305,2021-06-30,2021,6,10.377268,0.095515,10.106187,0.534640
306,2021-07-30,2021,7,10.153195,0.092964,10.235278,0.264689


In [193]:
df11.to_csv("eurotry.csv")

In [4]:
for i,j in zip(final_cust["FK_ADDRESS_COMMUNICATION_CITY"].value_counts().index,final_cust["FK_ADDRESS_COMMUNICATION_CITY"].value_counts()):
    print(i,"-----",j)

İstanbul ----- 271740
Ankara ----- 78830
İzmir ----- 39362
Antalya ----- 26717
Kocaeli ----- 15380
Bursa ----- 15258
Gaziantep ----- 14850
Konya ----- 14409
Adana ----- 13368
Kayseri ----- 12885
Mersin ----- 12669
Muğla ----- 10500
Trabzon ----- 9021
Hatay ----- 8607
Elazığ ----- 8076
Erzurum ----- 7992
Diyarbakır ----- 7960
Samsun ----- 7646
Eskişehir ----- 7455
Şanlıurfa ----- 7340
Tekirdağ ----- 6564
Sakarya ----- 6540
Denizli ----- 5815
Manisa ----- 5702
Balıkesir ----- 5687
Aydın ----- 5275
Malatya ----- 4727
Afyonkarahisar ----- 4425
Kahramanmaraş ----- 4414
Zonguldak ----- 4114
Van ----- 4096
Edirne ----- 3787
Isparta ----- 3504
Ordu ----- 3053
Kütahya ----- 2894
Çorum ----- 2881
Mardin ----- 2659
Nevşehir ----- 2567
Çanakkale ----- 2558
Tokat ----- 2498
Batman ----- 2438
Kırklareli ----- 2322
Yozgat ----- 2312
Giresun ----- 2305
Rize ----- 2170
Osmaniye ----- 2168
Sivas ----- 2094
Bolu ----- 2049
Düzce ----- 1963
Adıyaman ----- 1847
Amasya ----- 1818
Uşak ----- 1802
Ağrı ----- 

In [5]:
replace_for_cities = {'DÜZCE':'Düzce',
                      'BİNGÖL':'Bingöl',
                      'SİVAS':'Sivas',
                      'BİLİNMEYEN':'wronginfo',
                      'X':'wronginfo',
                      'Alm':'Almanya',
                      'KÜTAHYA':'Kütahya',
                      'bursa':'Bursa',
                      'ist':'İstanbul',
                      'L':'wronginfo',
                      'istanbul':'İstanbul',
                      'DİYARBAKIR':'Diyarbakır',
                      'DİĞER':'wronginfo',
                      'z':'wronginfo',
                      'nahçıvan':'Nahçıvan',
                      'RİZE':'Rize',
                      'girne':'GİRNE',
                      'Merkez':'wronginfo',
                      'Asd':'wronginfo',
                      'Lefkoşe':'Lefkoşa',
                      'z':'wronginfo'}

In [6]:
final_cust["FK_ADDRESS_COMMUNICATION_CITY"]=final_cust["FK_ADDRESS_COMMUNICATION_CITY"].replace(replace_for_cities)

In [7]:
final_cust[final_cust["BIRTH_DATE"]>2010]

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,GENDER_ID,MARITAL_STATUS,MARITAL_STATUS_ID,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION
2503,2503,189467,1849715,Erkek,100,NaN,0,2021.0,Bolu,NaN
5876,5876,176675,1845788,Erkek,100,NaN,0,2021.0,Antalya,NaN
7932,7932,135837,1840556,Erkek,100,NaN,0,2021.0,İstanbul,NaN
9008,9008,147157,1837065,Erkek,100,NaN,0,2020.0,Tekirdağ,NaN
13500,13500,185171,1834583,Erkek,100,NaN,0,2020.0,Muğla,NaN
...,...,...,...,...,...,...,...,...,...,...
836492,836492,195445,1841935,Erkek,100,NaN,0,2021.0,NaN,NaN
843894,843894,150447,1837922,Erkek,100,NaN,0,2020.0,Antalya,NaN
846319,846319,60339,1244773,Erkek,100,Bekar,102,2017.0,İstanbul,Diğer
847862,847862,193900,1828986,Erkek,100,NaN,0,2020.0,İstanbul,NaN


In [8]:
final_sales

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE
0,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0
1,1,1149729,4839430.0,2015-11-11 17:03:28,102,K,9184.0
2,2,1697993,4847398.0,2015-11-14 11:15:51,102,Y,9231.0
3,3,1680258,3817349.0,2014-12-23 17:26:09,105,Y,9362.0
4,4,1244973,8079074.0,2018-08-28 12:55:04,101,K,9901.0
...,...,...,...,...,...,...,...
397521,397521,1151001,3504117.0,2014-09-17 12:30:13,102,K,9435.0
397522,397522,1581191,3518578.0,2014-09-22 20:14:38,105,K,9176.0
397523,397523,1535801,3526258.0,2014-09-25 12:01:58,102,T,9352.0
397524,397524,1205995,6592609.0,2017-04-14 15:40:44,105,X,9523.0


In [9]:
# REQ_TOPMODEL_CODE Müşterinin talep ettiği araç modelini belirtir.
final_sales["REQ_TOPMODEL_CODE"].unique()

array([9231., 9184., 9362., 9901., 9245., 9419., 9352., 9176., 9612.,
       9821., 9511., 9094., 9377., 9390., 9596., 9387., 9524., 9346.,
       9095., 9355., 9936., 9123., 9186., 9527., 9766., 9625., 9817.,
       9522., 9935., 9161., 9640., 9927., 9349., 9526., 9819., 9435.,
       9525., 9692., 9172., 9816., 9351., 9119., 9222., 9298., 9520.,
       9503., 9311., 9235., 9822., 9820., 9201., 9357., 9345., 9430.,
       9374., 9174., 9721., 9605., 9515., 9523., 9581., 9818., 9604.,
       9930., 9763., 9241., 9516., 9200., 9531., 9199., 9438., 9517.,
       9929., 9295., 9359., 9898., 9940., 9767., 9764., 9815., 9931.,
       9431., 9518., 9739., 9281., 9928., 9967., 9969., 9968., 9233.,
       9897., 9434., 9204., 9765., 9358., 9902., 9301., 9626., 9694.,
       9361., 9183., 9509., 9354., 9464., 9336., 9899., 9313., 9823.,
       9907., 9521., 9510., 9240., 9243., 9232., 9533., 9505., 9512.,
       9616., 9312., 9495., 9133., 9083.,   nan, 9645., 9690., 9376.,
       9135., 9163.,

In [10]:
final_sales["REQ_BRAND_CODE"].unique()

array(['Y', 'K', 'Z', 'T', 'X'], dtype=object)

In [11]:
final_sales["STATUS"].unique()

array([101, 102, 105, 100, 109, 106], dtype=int64)

In [12]:
final_cust_related

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,END_DATE,FK_RELATION_STATUS_ID,FK_RELATION_STATUS_EXPLANATION
0,0,189537,5321560,2018-07-24,2019-07-03,2,Pasif Ruhsat Sahibi
1,1,110802,5329278,2019-07-16,NaN,1,Aktif Ruhsat Sahibi
2,2,160615,5329282,2019-09-12,NaN,1,Aktif Ruhsat Sahibi
3,3,115664,5328513,2019-02-25,NaN,1,Aktif Ruhsat Sahibi
4,4,81061,5322264,2018-04-16,NaN,1,Aktif Ruhsat Sahibi
...,...,...,...,...,...,...,...
374874,374874,154304,5179455,2010-10-19,NaN,1,Aktif Ruhsat Sahibi
374875,374875,75199,5180458,2019-01-02,NaN,1,Aktif Ruhsat Sahibi
374876,374876,9036,5177679,2012-05-28,NaN,1,Aktif Ruhsat Sahibi
374877,374877,33936,5172864,2014-08-16,NaN,1,Aktif Ruhsat Sahibi


In [13]:
df = pd.DataFrame(final_cust_related,
                  columns=['FK_RELATION_STATUS_ID', 'FK_RELATION_STATUS_EXPLANATION'])
df.corr()

,FK_RELATION_STATUS_ID
FK_RELATION_STATUS_ID,1.0


In [14]:
final_cust_related.isnull().sum()

Unnamed: 0                             0
BASE_CUSTOMER_ID                       0
VEHICLE_ID                             0
START_DATE                             0
END_DATE                          215122
FK_RELATION_STATUS_ID                  0
FK_RELATION_STATUS_EXPLANATION         0
dtype: int64

In [16]:
final_sifir_arac

,Unnamed: 0,VEHICLE_ID,CUSTOMER_ID,CREATE_DATE
0,0,5015193,1063740,2005-01-01 19:08:33
1,1,5110231,1155678,2005-09-29 17:16:15
2,2,5290356,1737831,2016-04-29 13:12:10
3,3,5344576,1841362,2021-03-16 13:39:01
4,4,5094010,1066916,2005-04-20 09:51:19
...,...,...,...,...
169520,169520,5160829,1269016,2009-06-10 16:16:15
169521,169521,5098542,1157563,2005-08-01 12:07:57
169522,169522,5148086,1156732,2007-12-28 17:15:09
169523,169523,5263144,1032834,2015-06-24 11:19:15


In [17]:
final_sifir_arac.isnull().sum()

Unnamed: 0     0
VEHICLE_ID     0
CUSTOMER_ID    0
CREATE_DATE    0
dtype: int64

In [18]:
final_vehicle

,Unnamed: 0,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE
0,0,5317713,2017-12-07,Z,13398,9349,Benzin,Manuel
1,1,5259212,2014-12-09,X,15694,9524,Dizel,Otomatik (DSG)
2,2,5110414,2005-12-12,Y,11183,9118,Kurşunsuz Benzin,Otomatik
3,3,5101263,2005-08-23,X,15039,9495,Benzin,Manuel
4,4,5053909,2003-04-09,Y,12255,9219,NaN,NaN
...,...,...,...,...,...,...,...,...
345087,345087,5253033,2014-07-24,T,13515,9352,Benzin,Manuel
345088,345088,5174679,2010-05-27,T,13833,9376,Benzin,Manuel
345089,345089,5059846,2004-12-14,X,15316,9509,Benzin,Otomatik
345090,345090,5080882,2004-06-29,K,14615,9446,Dizel,Manuel


In [36]:
final_vehicle.isnull().sum()

Unnamed: 0            0
VEHICLE_ID            0
TRAFFIC_DATE        866
BRAND_CODE            0
BASEMODEL_CODE        0
TOPMODEL_CODE         0
MOTOR_GAS_TYPE    72446
GEAR_BOX_TYPE     84015
dtype: int64

In [38]:
final_vehicle["GEAR_BOX_TYPE"].value_counts()

Manuel              149421
Otomatik             58702
Otomatik(DSG)        41679
Sürekli Değişken     11275
Name: GEAR_BOX_TYPE, dtype: int64

In [37]:
replace_for_gear_box_type = {'manuel':'Manuel',
                      'Otomatik (DSG)':'Otomatik(DSG)',
                      'Otomatik (DSG Şanzuman)':'Otomatik(DSG)',
                      'Otomaik':'Otomatik',
                      '?automatisch':'Otomatik',
                      'Sürekli değişken':'Sürekli Değişken',
                      'Otomatik Değişken':'Otomatik',
                      'Düz':'Manuel',
                      '?Handschaltung':'Manuel',
                      'Mekanik':'Manuel'
                     }
final_vehicle["GEAR_BOX_TYPE"]=final_vehicle["GEAR_BOX_TYPE"].replace(replace_for_gear_box_type)

In [31]:
replace_for_gas_type = {'?Benzin':'Benzin',
                      'Kurşunsuz benzin':'Kurşunsuz Benzin',
                      'Benzin/Hybrid':'Benzin/Hibrit',
                      'Diesel':'Dizel'
                     }
final_vehicle["MOTOR_GAS_TYPE"]=final_vehicle["MOTOR_GAS_TYPE"].replace(replace_for_gas_type)

In [20]:
mask_serv_bakim

,Unnamed: 0,CREATE_DATE,IS_MAINTENANCE,VEHICLE_ID,TOTAL_AMOUNT_TL
0,0,2010-12-29 10:50:56,1,5000001,192.0
1,1,2013-08-21 10:34:21,1,5000001,267.0
2,2,2015-11-30 10:55:00,1,5000001,1747.0
3,3,2015-12-09 13:38:49,0,5000001,465.0
4,4,2017-11-23 09:53:22,1,5000001,549.0
...,...,...,...,...,...
1040995,1040995,2021-05-20 14:58:31,0,5344976,1180.0
1040996,1040996,2021-05-24 09:57:34,0,5344981,0.0
1040997,1040997,2021-05-25 12:44:02,0,5344999,3258.0
1040998,1040998,2021-05-27 09:45:00,0,5345037,1.0


In [21]:
mask_serv_bakim.isnull().sum()

Unnamed: 0             0
CREATE_DATE            0
IS_MAINTENANCE         0
VEHICLE_ID             0
TOTAL_AMOUNT_TL    75404
dtype: int64

In [23]:
mask_serv_bakim["IS_MAINTENANCE"].value_counts()

0    558850
1    482150
Name: IS_MAINTENANCE, dtype: int64

In [ ]:
final_cust['GENDER_ID'].value_counts()

In [ ]:
final_cust['MARITAL_STATUS'].isna().sum()

In [ ]:
final_cust['MARITAL_STATUS_ID'].value_counts()

In [43]:
cust_sales

,Unnamed: 0_x,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,Unnamed: 0_y,BASE_CUSTOMER_ID,GENDER,GENDER_ID,MARITAL_STATUS,MARITAL_STATUS_ID,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION
0,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,374633,31024,Erkek,100,Evli,101,1971.0,Düzce,Diğer
1,1,1149729,4839430.0,2015-11-11 17:03:28,102,K,9184.0,520405,34723,Erkek,100,Evli,101,1975.0,Konya,Oto. Bayisi / Oto. Galerisi
2,2,1697993,4847398.0,2015-11-14 11:15:51,102,Y,9231.0,538247,133025,Erkek,100,Evli,101,1975.0,NaN,Oto. Tamir / Servis / Yedek Parça / Teknik Destek
3,3,1680258,3817349.0,2014-12-23 17:26:09,105,Y,9362.0,418136,188043,Erkek,100,NaN,0,NaN,NaN,NaN
4,4,1244973,8079074.0,2018-08-28 12:55:04,101,K,9901.0,315387,60402,Erkek,100,Bekar,102,1941.0,Ankara,Emekli
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397521,397521,1151001,3504117.0,2014-09-17 12:30:13,102,K,9435.0,667643,91981,Erkek,100,Evli,101,1972.0,Diyarbakır,Esnaf
397522,397522,1581191,3518578.0,2014-09-22 20:14:38,105,K,9176.0,16140,85131,Erkek,100,Evli,101,1975.0,Antalya,Serbest
397523,397523,1535801,3526258.0,2014-09-25 12:01:58,102,T,9352.0,756333,175291,Erkek,100,Evli,101,1969.0,Kocaeli,Bilişimci / Bilgisayar
397524,397524,1205995,6592609.0,2017-04-14 15:40:44,105,X,9523.0,223440,107878,Erkek,100,Evli,101,1979.0,İstanbul,NaN


In [67]:
sales_cust= pd.merge(final_sales, final_cust, on=['CUSTOMER_ID'], how='left')

In [68]:
related_cust = pd.merge(final_cust_related, final_cust, on=['BASE_CUSTOMER_ID'], how='left')

In [59]:
related_sifir_cust = pd.merge(related_sifir, final_cust, on=['CUSTOMER_ID'], how='left')

In [69]:
mask_final_vehicle = pd.merge(mask_serv_bakim, final_vehicle, on=['VEHICLE_ID'], how='left')

In [70]:
related_sifir_mask_veh = pd.merge(related_sifir_cust, mask_final_vehicle, on=['VEHICLE_ID'], how='left')

In [71]:
related_sifir_mask_veh_rel = pd.merge(final_cust_related, related_sifir_mask_veh, on=['VEHICLE_ID'], how='left')

In [ ]:
related_sifir_mask_veh_rel_sales = pd.merge(sales_cust, related_sifir_mask_veh, on=['BASE_CUSTOMER_ID'], how='left')

In [66]:
cust_sales_related.dropna()

,Unnamed: 0_x,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,Unnamed: 0_y,BASE_CUSTOMER_ID,GENDER,...,MARITAL_STATUS_ID,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION,Unnamed: 0,VEHICLE_ID,START_DATE,END_DATE,FK_RELATION_STATUS_ID,FK_RELATION_STATUS_EXPLANATION
2,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,374633,31024,Erkek,...,101,1971.0,Düzce,Diğer,297993.0,5315137.0,2017-12-12,2020-04-30,2.0,Pasif Ruhsat Sahibi
5,1,1149729,4839430.0,2015-11-11 17:03:28,102,K,9184.0,520405,34723,Erkek,...,101,1975.0,Konya,Oto. Bayisi / Oto. Galerisi,100427.0,5076084.0,2007-08-09,2010-09-29,2.0,Pasif Ruhsat Sahibi
6,1,1149729,4839430.0,2015-11-11 17:03:28,102,K,9184.0,520405,34723,Erkek,...,101,1975.0,Konya,Oto. Bayisi / Oto. Galerisi,181441.0,5099190.0,2005-06-23,2009-09-23,2.0,Pasif Ruhsat Sahibi
7,1,1149729,4839430.0,2015-11-11 17:03:28,102,K,9184.0,520405,34723,Erkek,...,101,1975.0,Konya,Oto. Bayisi / Oto. Galerisi,182351.0,5027696.0,2005-11-01,2021-02-19,2.0,Pasif Ruhsat Sahibi
26,4,1244973,8079074.0,2018-08-28 12:55:04,101,K,9901.0,315387,60402,Erkek,...,102,1941.0,Ankara,Emekli,104305.0,5084677.0,2005-03-11,2006-06-06,2.0,Pasif Ruhsat Sahibi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585315,397514,1428837,7355722.0,2017-12-09 14:06:00,102,X,9523.0,675915,137022,Erkek,...,101,1977.0,Konya,Esnaf,290372.0,5197571.0,2012-10-23,2015-04-28,2.0,Pasif Ruhsat Sahibi
1585329,397521,1151001,3504117.0,2014-09-17 12:30:13,102,K,9435.0,667643,91981,Erkek,...,101,1972.0,Diyarbakır,Esnaf,122249.0,5179152.0,2010-10-30,2011-12-27,2.0,Pasif Ruhsat Sahibi
1585336,397523,1535801,3526258.0,2014-09-25 12:01:58,102,T,9352.0,756333,175291,Erkek,...,101,1969.0,Kocaeli,Bilişimci / Bilgisayar,63695.0,5009269.0,2011-02-11,2012-06-02,2.0,Pasif Ruhsat Sahibi
1585341,397525,1501605,6614631.0,2017-04-22 11:42:48,102,K,9898.0,82983,139971,Erkek,...,101,1962.0,Kilis,Müteahit,236745.0,5272889.0,2016-12-23,2017-06-01,2.0,Pasif Ruhsat Sahibi


In [47]:
final_sifir_arac

,Unnamed: 0,VEHICLE_ID,CUSTOMER_ID,CREATE_DATE
0,0,5015193,1063740,2005-01-01 19:08:33
1,1,5110231,1155678,2005-09-29 17:16:15
2,2,5290356,1737831,2016-04-29 13:12:10
3,3,5344576,1841362,2021-03-16 13:39:01
4,4,5094010,1066916,2005-04-20 09:51:19
...,...,...,...,...
169520,169520,5160829,1269016,2009-06-10 16:16:15
169521,169521,5098542,1157563,2005-08-01 12:07:57
169522,169522,5148086,1156732,2007-12-28 17:15:09
169523,169523,5263144,1032834,2015-06-24 11:19:15


C:\Users\Salih\AppData\Local\Temp/ipykernel_12572/2615232290.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_y'} in the result is deprecated and will raise a MergeError in a future version.
  cust_sales_related_sifir = pd.merge(final_sifir_arac, cust_sales_related, on=['CUSTOMER_ID'], how='left')


In [48]:
cust_sales_related_sifir

,Unnamed: 0_x,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,Unnamed: 0_y,BASE_CUSTOMER_ID,GENDER,...,OCCUPATION,Unnamed: 0_x,VEHICLE_ID_x,START_DATE,END_DATE,FK_RELATION_STATUS_ID,FK_RELATION_STATUS_EXPLANATION,Unnamed: 0_y,VEHICLE_ID_y,CREATE_DATE
0,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,374633,31024,Erkek,...,Diğer,102956.0,5067245.0,2004-08-18,NaN,1.0,Aktif Ruhsat Sahibi,52348.0,5315137.0,2017-12-12 14:18:09
1,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,374633,31024,Erkek,...,Diğer,102956.0,5067245.0,2004-08-18,NaN,1.0,Aktif Ruhsat Sahibi,75297.0,5280177.0,2015-10-09 16:21:28
2,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,374633,31024,Erkek,...,Diğer,281601.0,5280177.0,2015-10-09,NaN,1.0,Aktif Ruhsat Sahibi,52348.0,5315137.0,2017-12-12 14:18:09
3,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,374633,31024,Erkek,...,Diğer,281601.0,5280177.0,2015-10-09,NaN,1.0,Aktif Ruhsat Sahibi,75297.0,5280177.0,2015-10-09 16:21:28
4,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,374633,31024,Erkek,...,Diğer,297993.0,5315137.0,2017-12-12,2020-04-30,2.0,Pasif Ruhsat Sahibi,52348.0,5315137.0,2017-12-12 14:18:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6611511,397524,1205995,6592609.0,2017-04-14 15:40:44,105,X,9523.0,223440,107878,Erkek,...,NaN,5701.0,5228762.0,2013-05-28,NaN,1.0,Aktif Ruhsat Sahibi,NaN,NaN,NaN
6611512,397524,1205995,6592609.0,2017-04-14 15:40:44,105,X,9523.0,223440,107878,Erkek,...,NaN,30265.0,5001081.0,2006-03-22,2011-01-03,2.0,Pasif Ruhsat Sahibi,NaN,NaN,NaN
6611513,397524,1205995,6592609.0,2017-04-14 15:40:44,105,X,9523.0,223440,107878,Erkek,...,NaN,208008.0,5183379.0,2011-01-10,2013-11-27,2.0,Pasif Ruhsat Sahibi,NaN,NaN,NaN
6611514,397525,1501605,6614631.0,2017-04-22 11:42:48,102,K,9898.0,82983,139971,Erkek,...,Müteahit,236745.0,5272889.0,2016-12-23,2017-06-01,2.0,Pasif Ruhsat Sahibi,NaN,NaN,NaN


In [52]:
mask_final_vehicle_sf_arac = pd.merge(mask_final_vehicle, final_sifir_arac, on=['VEHICLE_ID'], how='left')

In [ ]:
mask_final_vehicle_sf_arac = pd.merge(mask_final_vehicle, final_sifir_arac, on=['VEHICLE_ID'], how='left')

In [53]:
mask_final_vehicle_sf_arac_rel = pd.merge(mask_final_vehicle_sf_arac, final_cust_related , on=['VEHICLE_ID'], how='left')

C:\Users\Salih\AppData\Local\Temp/ipykernel_12572/705236525.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x'} in the result is deprecated and will raise a MergeError in a future version.
  mask_final_vehicle_sf_arac_rel = pd.merge(mask_final_vehicle_sf_arac, final_cust_related , on=['VEHICLE_ID'], how='left')


In [55]:
mask_final_vehicle_sf_arac_rel_1 = pd.merge(mask_final_vehicle_sf_arac, cust_sales , on=['BASE_CUSTOMER_ID'], how='left')

KeyError: 'BASE_CUSTOMER_ID'

In [ ]:
cust_related_sifir_arac = pd.merge(final_cust_related, final_sifir_arac, on=['VEHICLE_ID'], how='left')

In [ ]:
cust_related_sifir_arac

In [ ]:
cust_related_sifir_arac_final_cust = pd.merge(final_cust, cust_related_sifir_arac, on=['BASE_CUSTOMER_ID','CUSTOMER_ID'], how='left')

In [ ]:
mask_serv_bakim

In [ ]:
final_vehicle

In [ ]:
cust_related_sifir_arac_final_cust

In [ ]:
mask_final_vehicle